In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2

# Load the ASL dataset from Kaggle
DATASET_DIR = 'asl_alphabet_train/asl_alphabet_train'
CATEGORIES = sorted(os.listdir(DATASET_DIR))
IMG_SIZE = 64

def create_dataset():
    X = []
    y = []
    for category in CATEGORIES:
        path = os.path.join(DATASET_DIR, category)
        label = CATEGORIES.index(category)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label)
    X = np.array(X).astype('float32') / 255.0
    y = np.array(y)
    return X, y

X, y = create_dataset()

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(CATEGORIES), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_val, y_val, verbose=2)
print('Test accuracy:', test_acc)


KeyboardInterrupt: 

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2

In [2]:
# Save the model
MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

model.save(os.path.join(MODEL_DIR, 'asl_model.h5'))

NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the saved model
MODEL_PATH = 'model/asl_model.h5'
model = load_model(MODEL_PATH)

# Load a sample image for testing
img_path = 'test pic.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (64, 64))
img = np.array(img).astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

# Use the model to predict the class label of the image
pred = model.predict(img)
pred_label = np.argmax(pred)

# Print the predicted label
print('Predicted label:', CATEGORIES[pred_label])

In [2]:
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the saved model
MODEL_PATH = 'model/asl_model.h5'
model = load_model(MODEL_PATH)

# Load the ASL labels
CATEGORIES = sorted(os.listdir('asl_alphabet_train/asl_alphabet_train'))

# Load the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Flip the frame horizontally (mirror effect)
    frame = cv2.flip(frame, 1)

    # Draw a rectangle around the region of interest
    x, y, w, h = 100, 100, 300, 300
    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    # Extract the region of interest and preprocess it
    roi = frame[y:y+h, x:x+w]
    roi = cv2.resize(roi, (64, 64))
    roi = np.array(roi).astype('float32') / 255.0
    roi = np.expand_dims(roi, axis=0)

    # Use the model to predict the class label of the region of interest
    pred = model.predict(roi)
    pred_label = np.argmax(pred)

    # Add the predicted label to the frame
    label = CATEGORIES[pred_label]
    cv2.putText(frame, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    # Show the frame
    cv2.imshow('ASL Recognition', frame)

    # Exit the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()